In [4]:
"""
Clean TradingView 15‑minute export for QQQ
-----------------------------------------
▶ Assumes RTH‑only export and VWAP anchored 09:30 2‑Jan‑2025.
▶ Adds `ST_dir` (+1 / ‑1) alongside the numeric SuperTrend band.
"""

import pandas as pd
from pathlib import Path
import pytz

# ── 1. CONFIG ─────────────────────────────────────────────────────────────
SRC_CSV = Path(r"C:\Users\mason\Downloads\BATS_QQQ, 15 (2).csv")
DST_CSV = SRC_CSV.with_name("QQQ_15m_YTD_clean_ST.csv")
ANCHOR_DT = "2025-01-02 09:30"   # first RTH bar of 2025 (Eastern)

# ── 2. LOAD & MAKE DATETIME INDEX ─────────────────────────────────────────
ny = pytz.timezone("America/New_York")

df = pd.read_csv(SRC_CSV)
df["dt"] = (pd.to_datetime(df["time"], unit="s", utc=True)
              .dt.tz_convert(ny))

df = (df.set_index("dt")
        .sort_index())

# ── 3. SLICE TO YTD ───────────────────────────────────────────────────────
df = df.loc[ANCHOR_DT:]

# ── 4. DROP 100 %‑NaN & LABEL COLUMNS ─────────────────────────────────────
label_cols = df.filter(regex=r"^(Plot|Regular)").columns
all_nan    = df.columns[df.isna().all()]
df = df.drop(columns=label_cols.union(all_nan))

# ── 5. BUILD SUPERTREND BAND & DIRECTION FLAG ─────────────────────────────
if {"Up Trend", "Down Trend"}.issubset(df.columns):
    df["SuperTrend"] = df["Up Trend"].fillna(df["Down Trend"])
    df = df.drop(columns=["Up Trend", "Down Trend"])

# **NEW: create +1 / ‑1 direction column**
df["ST_dir"] = (df["close"] > df["SuperTrend"]).astype(int).replace({0: -1})

# ── 6. OPTIONAL: RENAME KELTNER CHANNEL COLUMNS ──────────────────────────
kc_map = {"Upper": "KC_Upper", "Basis": "KC_Mid", "Lower": "KC_Lower"}
df = df.rename(columns={k: v for k, v in kc_map.items() if k in df.columns})

# ── 7. SAVE ───────────────────────────────────────────────────────────────
df.to_csv(DST_CSV, index_label="datetime")
print(f"✅  Saved cleaned file with {df.shape[0]:,} rows × {df.shape[1]} columns → {DST_CSV}")


✅  Saved cleaned file with 1,898 rows × 32 columns → C:\Users\mason\Downloads\QQQ_15m_YTD_clean_ST.csv


In [7]:
import pandas as pd
from pathlib import Path
import pytz

# ── 1. FILE LOCATIONS ────────────────────────────────────────────────────
QQQ_CSV = Path(r"C:\Users\mason\Downloads\QQQ_15m_YTD_clean_ST.csv")
VIX_CSV = Path(r"C:\Users\mason\Downloads\CBOE_DLY_VIX, 15.csv")
OUT_CSV = QQQ_CSV.with_name("QQQ_15m_YTD_clean_ST_VIX.csv")

NY = pytz.timezone("America/New_York")
ANCHOR_DT = "2025-01-02 09:30"

# ── 2. LOAD QQQ (EXPLICIT DATETIME PARSE) ────────────────────────────────
qqq = pd.read_csv(QQQ_CSV)

# force datetime parse, assume strings like "2025‑01‑02 09:30:00‑05:00"
qqq["datetime"] = (
    pd.to_datetime(qqq["datetime"], utc=True)   # parse & normalize to UTC
        .dt.tz_convert(NY)                     # convert to America/New_York
)
qqq = qqq.set_index("datetime").sort_index()

# ── 3. LOAD VIX & PREPARE ────────────────────────────────────────────────
vix = pd.read_csv(VIX_CSV)
vix["datetime"] = (
    pd.to_datetime(vix["time"], unit="s", utc=True)
        .dt.tz_convert(NY)
)
vix = (
    vix.loc[:, ["datetime", "close"]]
       .rename(columns={"close": "VIX"})
       .set_index("datetime")
       .sort_index()
       .loc[ANCHOR_DT:]          # keep YTD only
)

# ── 4. JOIN & FILL ───────────────────────────────────────────────────────
merged = (
    qqq.join(vix, how="left")
        .ffill()                 # forward‑fill tiny gaps
)

# ── 5. SAVE ──────────────────────────────────────────────────────────────
merged.to_csv(OUT_CSV, index_label="datetime")
print(f"✅  Added VIX column → {OUT_CSV}")
print(f"   Final shape: {merged.shape[0]:,} rows × {merged.shape[1]} columns")


✅  Added VIX column → C:\Users\mason\Downloads\QQQ_15m_YTD_clean_ST_VIX.csv
   Final shape: 1,898 rows × 33 columns


In [8]:
"""
Append new community‑indicator columns to the cleaned QQQ 15‑minute dataset
---------------------------------------------------------------------------
"""

import pandas as pd
from pathlib import Path
import pytz

# ── 1. FILE LOCATIONS ────────────────────────────────────────────────────
RAW_CSV   = Path(r"C:\Users\mason\Downloads\BATS_QQQ, 15 (3).csv")  # new export
BASE_CSV  = Path(r"C:\Users\mason\Downloads\QQQ_15m_YTD_clean_ST_VIX.csv")
OUT_CSV   = BASE_CSV  # overwrite; change .with_name("…_new.csv") if you want a copy

NY        = pytz.timezone("America/New_York")
ANCHOR_DT = "2025-01-02 09:30"   # first 15‑min RTH bar of 2025

# ── 2. LOAD BASE DATASET ────────────────────────────────────────────────
base = pd.read_csv(BASE_CSV)
base["datetime"] = (pd.to_datetime(base["datetime"], utc=True)
                      .dt.tz_convert(NY))
base = base.set_index("datetime")

# ── 3. LOAD RAW EXPORT & CLEAN ──────────────────────────────────────────
raw = pd.read_csv(RAW_CSV)

raw["datetime"] = (pd.to_datetime(raw["time"], unit="s", utc=True)
                     .dt.tz_convert(NY))
raw = (raw
        .set_index("datetime")
        .sort_index()
        .loc[ANCHOR_DT:])                     # keep YTD range only

# drop 100 %‑NaN columns & label/plot text columns
label_cols = raw.filter(regex=r"^(Plot|Regular|Label|Dot)").columns
all_nan    = raw.columns[raw.isna().all()]
raw = raw.drop(columns=label_cols.union(all_nan))

# ── 4. KEEP ONLY *NEW* COLUMNS ───────────────────────────────────────────
new_cols = [c for c in raw.columns if c not in base.columns]
new_data = raw[new_cols]

# ── 5. MERGE & SAVE ──────────────────────────────────────────────────────
merged = base.join(new_data, how="left")

merged.to_csv(OUT_CSV, index_label="datetime")
print(f"✅  Added {len(new_cols)} new column(s) → {OUT_CSV}")
print("   New columns:", ", ".join(new_cols))


✅  Added 18 new column(s) → C:\Users\mason\Downloads\QQQ_15m_YTD_clean_ST_VIX.csv
   New columns: Long Stop, Long Stop Start, Buy Label, Short Stop, Short Stop Start, Sell Label, Secondary QQE Trend Line, Secondary RSI Histogram, QQE Up Signal, QQE Down Signal, Percentile Rank Columns, Extreme Move Percentile Threshold Line, Warning Percentile Threshold Line, Percentile Rank Columns.1, Extreme Move Percentile Threshold Line.1, Warning Percentile Threshold Line.1, 0 Line Circles Plot, 0 Line-Line Plot
